## **Atención Q, K y V**

Con el objetivo de entender con detalle el funcionamiento de la atención en transformers, implementaremos una versión simplificada de la atención con los vectores Q, K y V.

In [62]:
import torch

# Supongamos que tenemos los siguiente vectores Q, K y V
Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

print(Q.shape)
print(K.shape)
print(V.shape)

score = Q @ K.transpose(0, 1)  # @ equivale a la multiplicación de matrices

print("\nScore:\n", score)

score = score / torch.sqrt(torch.tensor(K.shape[1]).float())  # Dividimos por la raíz cuadrada de la dimensión de K
score = torch.softmax(score, dim=1)

Z = score @ V  

print("\nResultado:\n", Z)

torch.Size([4, 3])
torch.Size([4, 3])
torch.Size([4, 3])

Score:
 tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.3000, 0.6000, 0.9000, 1.2000],
        [0.0600, 0.1200, 0.1800, 0.2400],
        [0.0900, 0.1800, 0.2700, 0.3600]])

Resultado:
 tensor([[0.2500, 0.5000, 0.5000],
        [0.1892, 0.5432, 0.5857],
        [0.2372, 0.5087, 0.5173],
        [0.2309, 0.5130, 0.5260]])


### **Todo en uno: scaled_dot_product_attention**

Todo el cálculo anterior lo realiza eficientemente una función de PyTorch llamada **scaled_dot_product_attention()**. Esta función calcula la atención en los tensores de consulta (query), clave (key) y valor (value), utilizando una máscara de atención opcional si se proporciona, y aplicando *dropout* si se especifica una probabilidad mayor a 0.0.

https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html

In [56]:
from torch.nn.functional import scaled_dot_product_attention

Z = scaled_dot_product_attention(Q, K, V)

print(Z)

tensor([[0.2500, 0.5000, 0.5000],
        [0.1892, 0.5432, 0.5857],
        [0.2372, 0.5087, 0.5173],
        [0.2309, 0.5130, 0.5260]])


## **Atención Q, K y V con enmascaramiento**

El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

In [4]:
import torch

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

score = Q @ K.transpose(0, 1)
#print(score)
score = score / torch.sqrt(torch.tensor(K.shape[1]).float())
print(score)

tril = torch.tril(torch.ones(score.shape[0], score.shape[0]))  # Creamos la máscara
score_masked = score.masked_fill(tril == 0, float('-inf'))  # Aplicamos la máscara al score. Todo lo que sea 0 en la máscara, lo reemplazamos por -inf para que al aplicar la softmax, se vuelva 0
print(score_masked)
score_masked = torch.softmax(score_masked, dim=1)
Z = score_masked @ V  

print(Z)

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.1732, 0.3464, 0.5196, 0.6928],
        [0.0346, 0.0693, 0.1039, 0.1386],
        [0.0520, 0.1039, 0.1559, 0.2078]])
tensor([[0.0000,   -inf,   -inf,   -inf],
        [0.1732, 0.3464,   -inf,   -inf],
        [0.0346, 0.0693, 0.1039,   -inf],
        [0.0520, 0.1039, 0.1559, 0.2078]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


Vemos de nuevo que podemos hacer lo mismo con la función **torch.nn.functional.scaled_dot_product_attention()** estableciendo el parámetro **is_causal** a True.

In [54]:
Z = scaled_dot_product_attention(Q, K, V, is_causal=True)

print(Z)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])
